# 論文 4：遞迴神經網路正則化（Recurrent Neural Network Regularization）
## Wojciech Zaremba, Ilya Sutskever, Oriol Vinyals (2014)

### RNN 的 Dropout

關鍵洞察：**僅對非遞迴連接應用 dropout**，不對遞迴連接使用。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 標準 Dropout

In [ ]:
def dropout(x, dropout_rate=0.5, training=True):
    """
    標準 dropout
    訓練時：以 dropout_rate 的機率隨機將元素歸零
    測試時：乘以 (1 - dropout_rate) 進行縮放
    """
    if not training or dropout_rate == 0:
        return x
    
    # 反向 dropout（訓練時縮放）
    mask = (np.random.rand(*x.shape) > dropout_rate).astype(float)
    return x * mask / (1 - dropout_rate)

# 測試 dropout
x = np.ones((5, 1))
print("原始值：", x.T)
print("加入 dropout (p=0.5)：", dropout(x, 0.5).T)
print("加入 dropout (p=0.5)：", dropout(x, 0.5).T)
print("測試模式：", dropout(x, 0.5, training=False).T)

## 具有正確 Dropout 的 RNN

**關鍵**：在**輸入**和**輸出**上使用 dropout，**不**在遞迴連接上使用！

In [ ]:
class RNNWithDropout:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 權重
        self.W_xh = np.random.randn(hidden_size, input_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = np.random.randn(output_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
    
    def forward(self, inputs, dropout_rate=0.0, training=True):
        """
        帶有 dropout 的前向傳遞
        
        Dropout 應用於：
        1. 輸入連接 (x -> h)
        2. 輸出連接 (h -> y)
        
        不應用於：
        - 遞迴連接 (h -> h)
        """
        h = np.zeros((self.hidden_size, 1))
        outputs = []
        hidden_states = []
        
        for x in inputs:
            # 對輸入應用 dropout
            x_dropped = dropout(x, dropout_rate, training)
            
            # RNN 更新（遞迴連接不使用 dropout）
            h = np.tanh(
                np.dot(self.W_xh, x_dropped) +  # 這裡有 Dropout
                np.dot(self.W_hh, h) +           # 這裡沒有 Dropout
                self.bh
            )
            
            # 在輸出前對隱藏狀態應用 dropout
            h_dropped = dropout(h, dropout_rate, training)
            
            # 輸出
            y = np.dot(self.W_hy, h_dropped) + self.by  # 這裡有 Dropout
            
            outputs.append(y)
            hidden_states.append(h)
        
        return outputs, hidden_states

# 測試
rnn = RNNWithDropout(input_size=10, hidden_size=20, output_size=10)
test_inputs = [np.random.randn(10, 1) for _ in range(5)]

outputs_train, _ = rnn.forward(test_inputs, dropout_rate=0.5, training=True)
outputs_test, _ = rnn.forward(test_inputs, dropout_rate=0.5, training=False)

print(f"訓練輸出[0] 平均值：{outputs_train[0].mean():.4f}")
print(f"測試輸出[0] 平均值：{outputs_test[0].mean():.4f}")

## 變分 Dropout（Variational Dropout）

**關鍵創新**：在所有時間步使用**相同的** dropout 遮罩！

In [ ]:
class RNNWithVariationalDropout:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 權重（與之前相同）
        self.W_xh = np.random.randn(hidden_size, input_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = np.random.randn(output_size, hidden_size) * 0.01
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
    
    def forward(self, inputs, dropout_rate=0.0, training=True):
        """
        變分 dropout：所有時間步使用相同遮罩
        """
        h = np.zeros((self.hidden_size, 1))
        outputs = []
        hidden_states = []
        
        # 為整個序列生成一次遮罩
        if training and dropout_rate > 0:
            input_mask = (np.random.rand(self.input_size, 1) > dropout_rate).astype(float) / (1 - dropout_rate)
            hidden_mask = (np.random.rand(self.hidden_size, 1) > dropout_rate).astype(float) / (1 - dropout_rate)
        else:
            input_mask = np.ones((self.input_size, 1))
            hidden_mask = np.ones((self.hidden_size, 1))
        
        for x in inputs:
            # 對每個輸入應用相同遮罩
            x_dropped = x * input_mask
            
            # RNN 更新
            h = np.tanh(
                np.dot(self.W_xh, x_dropped) +
                np.dot(self.W_hh, h) +
                self.bh
            )
            
            # 對每個隱藏狀態應用相同遮罩
            h_dropped = h * hidden_mask
            
            # 輸出
            y = np.dot(self.W_hy, h_dropped) + self.by
            
            outputs.append(y)
            hidden_states.append(h)
        
        return outputs, hidden_states

# 測試變分 dropout
var_rnn = RNNWithVariationalDropout(input_size=10, hidden_size=20, output_size=10)
outputs_var, _ = var_rnn.forward(test_inputs, dropout_rate=0.5, training=True)

print("變分 dropout 在所有時間步使用一致的遮罩")

## 比較 Dropout 策略

In [ ]:
# 生成合成序列資料
seq_length = 20
test_sequence = [np.random.randn(10, 1) for _ in range(seq_length)]

# 使用不同策略運行
_, h_no_dropout = rnn.forward(test_sequence, dropout_rate=0.0, training=False)
_, h_standard = rnn.forward(test_sequence, dropout_rate=0.5, training=True)
_, h_variational = var_rnn.forward(test_sequence, dropout_rate=0.5, training=True)

# 轉換為陣列
h_no_dropout = np.hstack([h.flatten() for h in h_no_dropout]).T
h_standard = np.hstack([h.flatten() for h in h_standard]).T
h_variational = np.hstack([h.flatten() for h in h_variational]).T

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].imshow(h_no_dropout, cmap='RdBu', aspect='auto')
axes[0].set_title('無 Dropout')
axes[0].set_xlabel('隱藏單元')
axes[0].set_ylabel('時間步')

axes[1].imshow(h_standard, cmap='RdBu', aspect='auto')
axes[1].set_title('標準 Dropout（每個時間步不同遮罩）')
axes[1].set_xlabel('隱藏單元')
axes[1].set_ylabel('時間步')

axes[2].imshow(h_variational, cmap='RdBu', aspect='auto')
axes[2].set_title('變分 Dropout（所有時間步相同遮罩）')
axes[2].set_xlabel('隱藏單元')
axes[2].set_ylabel('時間步')

plt.tight_layout()
plt.show()

print("變分 dropout 顯示一致的模式（相同單元在整個過程中被丟棄）")

## Dropout 位置很重要！

In [ ]:
# 視覺化 dropout 應用位置
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 建立簡單的 RNN 圖示
def draw_rnn_cell(ax, title, show_input_dropout, show_hidden_dropout, show_recurrent_dropout):
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # 繪製方塊
    # 輸入
    ax.add_patch(plt.Rectangle((1, 2), 1.5, 1, fill=True, color='lightblue', ec='black'))
    ax.text(1.75, 2.5, 'x_t', ha='center', va='center', fontsize=10)
    
    # 隱藏狀態（當前）
    ax.add_patch(plt.Rectangle((4, 4.5), 2, 2, fill=True, color='lightgreen', ec='black'))
    ax.text(5, 5.5, 'h_t', ha='center', va='center', fontsize=12)
    
    # 隱藏狀態（前一個）
    ax.add_patch(plt.Rectangle((7, 4.5), 2, 2, fill=True, color='lightyellow', ec='black'))
    ax.text(8, 5.5, 'h_{t-1}', ha='center', va='center', fontsize=10)
    
    # 輸出
    ax.add_patch(plt.Rectangle((4, 7.5), 2, 1, fill=True, color='lightcoral', ec='black'))
    ax.text(5, 8, 'y_t', ha='center', va='center', fontsize=10)
    
    # 箭頭
    # 輸入到隱藏
    color_input = 'red' if show_input_dropout else 'black'
    width_input = 3 if show_input_dropout else 1
    ax.arrow(2.5, 2.5, 1.3, 2, head_width=0.3, color=color_input, lw=width_input)
    if show_input_dropout:
        ax.text(3.2, 3.5, 'DROPOUT', fontsize=8, color='red', fontweight='bold')
    
    # 遞迴
    color_rec = 'red' if show_recurrent_dropout else 'black'
    width_rec = 3 if show_recurrent_dropout else 1
    ax.arrow(7, 5.5, -0.8, 0, head_width=0.3, color=color_rec, lw=width_rec)
    if show_recurrent_dropout:
        ax.text(6.5, 6.2, 'DROPOUT', fontsize=8, color='red', fontweight='bold')
    
    # 隱藏到輸出
    color_hidden = 'red' if show_hidden_dropout else 'black'
    width_hidden = 3 if show_hidden_dropout else 1
    ax.arrow(5, 6.6, 0, 0.7, head_width=0.3, color=color_hidden, lw=width_hidden)
    if show_hidden_dropout:
        ax.text(5.5, 7, 'DROPOUT', fontsize=8, color='red', fontweight='bold')

# 錯誤：到處都用 dropout
draw_rnn_cell(axes[0, 0], '錯誤：到處都用 Dropout\n（破壞時序流動）', 
             show_input_dropout=True, show_hidden_dropout=True, show_recurrent_dropout=True)

# 錯誤：只在遞迴連接
draw_rnn_cell(axes[0, 1], '錯誤：只在遞迴連接\n（失去梯度流動）', 
             show_input_dropout=False, show_hidden_dropout=False, show_recurrent_dropout=True)

# 正確：Zaremba 等人的方法
draw_rnn_cell(axes[1, 0], '正確：Zaremba 等人\n（僅輸入和輸出）', 
             show_input_dropout=True, show_hidden_dropout=True, show_recurrent_dropout=False)

# 無 dropout
draw_rnn_cell(axes[1, 1], '基準：無 Dropout\n（可能過擬合）', 
             show_input_dropout=False, show_hidden_dropout=False, show_recurrent_dropout=False)

plt.tight_layout()
plt.show()

## 關鍵要點

### 問題：
- 在 RNN 上使用樸素 dropout 效果不好
- 丟棄遞迴連接會破壞時序資訊流動
- 標準 dropout 每個時間步都改變遮罩（有雜訊）

### Zaremba 等人的解決方案：

**應用 dropout 於：**
- ✅ 輸入到隱藏的連接 (W_xh)
- ✅ 隱藏到輸出的連接 (W_hy)

**不應用於：**
- ❌ 遞迴連接 (W_hh)

### 變分 Dropout：
- 所有時間步使用**相同的 dropout 遮罩**
- 比改變遮罩更穩定
- 有更好的理論依據（貝葉斯）

### 結果：
- 語言建模有顯著改進
- Penn Treebank：測試困惑度從 78.4 改善到 68.7
- 也適用於 LSTM 和 GRU

### 實作技巧：
1. 使用比前饋網路更高的 dropout 率 (0.5-0.7)
2. 對雙向 RNN 在**兩個**方向都應用 dropout
3. 可以堆疊多個 LSTM 層，在層間使用 dropout
4. 變分 dropout：每個序列只生成一次遮罩

### 為什麼有效：
- 保持時序依賴（遞迴不使用 dropout）
- 正則化非時序轉換
- 強制對缺失輸入特徵的魯棒性
- 一致的遮罩（變分）減少變異